In [1]:
import pandas as pd
import numpy as np
import math
import time
import datetime


import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [2]:
path_train = 'train.csv'
path_test = 'test.csv'
path_test_out = 'model/'

In [3]:
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
print(train.shape)
print(test.shape)

(69306, 10)
(6400, 10)


In [4]:
train.isnull().sum()

TERMINALNO    0
TIME          0
TRIP_ID       0
LONGITUDE     0
LATITUDE      0
DIRECTION     0
HEIGHT        0
SPEED         0
CALLSTATE     0
Y             0
dtype: int64

In [5]:
def label_process(data):
    pre_label = data.drop_duplicates()
    return pre_label['Y'].values
pre_label = label_process(train[["TERMINALNO","Y"]])
pre_label.shape

(100,)

In [6]:
train.head()

,TERMINALNO,TIME,TRIP_ID,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE,Y
0,1,1476923580,1,122.985168,41.103741,12,39.402588,2.15,0,0.0
1,1,1476923640,1,122.984398,41.104904,24,39.311157,4.11,0,0.0
2,1,1476923700,1,122.986496,41.106388,74,34.178955,2.99,0,0.0
3,1,1476923760,1,122.989769,41.106884,115,37.765381,7.59,0,0.0
4,1,1476923820,1,122.991089,41.105442,151,36.049194,0.24,0,0.0


In [7]:
train = train.drop('Y',axis=1)

In [8]:
# feature = pd.concat([train,test],keys=['train','test'])
# feature.head()

In [9]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

In [10]:
def conver_time(data):
    data['Conver_TIME'] = data.TIME.apply(timestamp_datetime)
#     data['month'] = data.Conver_TIME.apply(lambda x: int(x[5:7]))
    data['hour'] = data.Conver_TIME.apply(lambda x: int(x[11:13]))
#     data = data.drop('TIME',axis=1)
#     data = data.drop('Conver_TIME',axis=1)
    return data

In [11]:
train = conver_time(train)
test = conver_time(test)

In [12]:
# train = train.sort_values(["TIME"])
# train.head(50)

In [13]:
def feature_process(data):
    set_data = set(data['TERMINALNO'])
    columns=['p_id',
             'maxTime',
             'phonerisk',
             'dir_risk',
             'height_risk',
             'speed_max',
             'speed_mean',
             'speed_var',
             'height_max',
             'height_mean',
             'height_var',
             'sp_he_mean',
             'zao',
             'wan',
             'shenye',
             'weizhi_ratio',
             'huchu_ratio',
             'huru_ratio',
             'liantong_ratio',
             'duanlian_ratio',
#              'call_spped0',
#              'call_spped1',
#              'call_spped2',
#              'call_spped3',
#              'call_spped4',
            ]
    feature = pd.DataFrame(columns=columns)
    
    # 针对每个用户进行分析
    for p_id in set_data:
        tempData = data.loc[data['TERMINALNO'] == p_id]
        tempData = tempData.sort_values(["TIME"])

        tempTime = tempData["TIME"].iloc[0]
        tempSpeed = tempData["SPEED"].iloc[0]
        tempDir = tempData["DIRECTION"].iloc[0]
        tempHeight = tempData["HEIGHT"].iloc[0]
        
        # 根据时间信息判断最长时间
        maxTime = 0
        maxTimelist = []

        # 用户行驶过程中，打电话危机上升
        phonerisk = 0

        # Direction 突变超过
        dir_risk = 0

        # Height 高度的危险值
        height_risk = 0
        zao=0
        wan=0
        shenye=0
        
        weizhi = 0
        huchu = 0
        huru = 0
        liantong = 0
        duanlian = 0
        
        for index, row in tempData.iterrows():
            
            hour = row['hour']
            if 7 <= hour <= 9:
                zao = 1
            elif 17 <= hour <= 19:
                wan = 1
            elif 0 <= hour < 7:
                shenye = 1

            if tempSpeed > 0 and row['CALLSTATE'] != 4:
                if row["CALLSTATE"] == 0:
                    phonerisk += math.exp(tempSpeed / 10) * 0.02
                else:
                    phonerisk += math.exp(tempSpeed / 10)
       
            if row["TIME"] - tempTime == 60:
                maxTime += 60
                tempTime = row["TIME"]

                # 判断方向变化程度与具有车速之间的危险系数
                dir_change = (min(abs(row["DIRECTION"] - tempDir), abs(360 + tempDir - row["DIRECTION"])) / 90.0)
                if tempSpeed != 0 and row["SPEED"] > 0:
                    dir_risk += math.pow((row["SPEED"] / 10), dir_change)
                
                # 海拔变化大的情况下和速度的危险系数
                height_risk += math.pow(abs(row["SPEED"] - tempSpeed) / 10,(abs(row["HEIGHT"] - tempHeight) / 100))
                
                tempHeight = row["HEIGHT"]

            elif row["TIME"] - tempTime > 60:
                maxTimelist.append(maxTime)
                maxTime = 0
                tempTime = row["TIME"]

                tempDir = row["DIRECTION"]
                tempHeight = row["HEIGHT"]
                tempSpeed = row["SPEED"]
                
            if row["CALLSTATE"] == 0:
                weizhi += 1
            elif row["CALLSTATE"] == 1:
                huchu += 1
            elif row["CALLSTATE"] == 2:
                huru += 1
            elif row["CALLSTATE"] == 3:
                liantong += 1
            elif row["CALLSTATE"] == 4:
                duanlian += 1
                

#         call0_spped = call_speed.loc[0]
#         call1_spped = call_speed.loc[1]
#         call2_spped = call_speed.loc[2]
#         call3_spped = call_speed.loc[3]
#         call4_spped = call_speed.loc[4]

        speed_max = tempData["SPEED"].max()
        speed_mean = tempData["SPEED"].mean()
        speed_var = tempData["SPEED"].var()
        
        height_max = tempData["HEIGHT"].max()
        height_mean = tempData["HEIGHT"].mean()
        height_var = tempData['HEIGHT'].var()
        
        sp_he_mean = speed_mean * height_mean

        maxTimelist.append(maxTime)
        maxTime = max(maxTimelist)
        
        total_callstate = len(tempData["CALLSTATE"])
        weizhi_ratio = weizhi / float(total_callstate)
        huchu_ratio = huchu / float(total_callstate)
        huru_ratio = huru / float(total_callstate)
        liantong_ratio = liantong / float(total_callstate)
        duanlian_ratio = duanlian / float(total_callstate)
        
        tempfeature = pd.DataFrame({'p_id':p_id,
                                    'maxTime':maxTime,
                                    'phonerisk':phonerisk,
                                    'dir_risk':dir_risk,
                                    'height_risk':height_risk,
                                    'speed_max':speed_max,
                                    'speed_mean':speed_mean,
                                    'speed_var':speed_var,
                                    'height_max':height_max,
                                    'height_mean':height_mean,
                                    'height_var':height_var,
                                    'sp_he_mean':sp_he_mean,
#                                     'call0_spped':call0_spped,
#                                     'call0_spped':call0_spped,
#                                     'call2_spped':call2_spped,
#                                     'call3_spped':call3_spped,
#                                     'call4_spped':call4_spped,
                                    'zao':zao,
                                    'wan':wan,
                                    'shenye':shenye,
                                    'weizhi_ratio':weizhi_ratio,
                                    'huchu_ratio':huchu_ratio,
                                    'huru_ratio':huru_ratio,
                                    'liantong_ratio':liantong_ratio,
                                    'duanlian_ratio':duanlian_ratio
                                    },
                                    index=['0'],
                                    columns=columns
                                    )
        
#         CALLSTATE_SET = set(tempData['CALLSTATE'])
#         call_speed = tempData["SPEED"].groupby(tempData['CALLSTATE']).mean()
#         for call_set in CALLSTATE_SET:
#             tempfeature['call_spped'+str(call_set)] = call_speed.loc[call_set]
            
        feature = feature.append(tempfeature,ignore_index=True)

        
    # feature = feature.values
    return feature

In [14]:
feature_train = feature_process(train)
feature_train = feature_train.fillna(0)

feature_test = feature_process(test)
feature_test = feature_test.fillna(0)

In [15]:
type(feature_train)

pandas.core.frame.DataFrame

In [16]:
print(feature_train.shape)
print(feature_test.shape)

(100, 20)
(100, 20)


In [17]:
feature_train.head()

,p_id,maxTime,phonerisk,dir_risk,height_risk,speed_max,speed_mean,speed_var,height_max,height_mean,height_var,sp_he_mean,zao,wan,shenye,weizhi_ratio,huchu_ratio,huru_ratio,liantong_ratio,duanlian_ratio
0,1.0,3540.0,22.698271,759.505736,253.833059,32.779999,17.489840,138.297543,224.069580,47.848093,907.782777,836.855477,1.0,1.0,1.0,0.489796,0.000000,0.000000,0.003401,0.506803
1,2.0,3180.0,67.556593,3646.161361,376.753980,36.119999,9.287734,153.238930,526.300537,71.400718,10543.556290,663.150842,0.0,1.0,1.0,0.396978,0.000000,0.000000,0.038462,0.564560
2,3.0,3600.0,41.291745,597.444549,840.111283,25.440001,7.987331,37.641736,125.748291,43.942672,295.097428,350.984677,1.0,1.0,0.0,0.870197,0.000000,0.001038,0.000000,0.128764
3,4.0,2700.0,110.310492,659.554444,656.575946,33.310001,6.312753,39.532636,115.885498,31.659250,165.859123,199.857010,1.0,1.0,0.0,0.228535,0.007576,0.003788,0.008838,0.751263
4,5.0,5520.0,39.709307,616.796195,776.837669,53.480000,7.695846,67.545555,117.702576,29.461915,207.698538,226.734356,1.0,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
feature_train.columns

Index(['p_id', 'maxTime', 'phonerisk', 'dir_risk', 'height_risk', 'speed_max',
       'speed_mean', 'speed_var', 'height_max', 'height_mean', 'height_var',
       'sp_he_mean', 'zao', 'wan', 'shenye', 'weizhi_ratio', 'huchu_ratio',
       'huru_ratio', 'liantong_ratio', 'duanlian_ratio', 'call_spped0',
       'call_spped1', 'call_spped2', 'call_spped3', 'call_spped4'],
      dtype='object')

In [18]:
#########################################################################################################################
# Training Data

In [19]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [20]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                               max_depth=4, max_features='sqrt',
                               min_samples_leaf=15, min_samples_split=10, 
                               loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                         learning_rate=0.05, max_depth=3, 
                         min_child_weight=1.7817, n_estimators=2200,
                         reg_alpha=0.4640, reg_lambda=0.8571,
                         subsample=0.5213, silent=1,
                         nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                          learning_rate=0.05, n_estimators=720,
                          max_bin = 55, bagging_fraction = 0.8,
                          bagging_freq = 5, feature_fraction = 0.2319,
                          feature_fraction_seed=9, bagging_seed=9,
                          min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                             meta_model = lasso)

In [21]:
feature_train = feature_train.values
X_TRAIN = feature_train[:,1:]
Y_TRAIN = pre_label
feature_test = feature_test.values
X_TEST = feature_test[:,1:]

In [23]:
stacked_averaged_models.fit(X_TRAIN, Y_TRAIN)
stacked_pred = stacked_averaged_models.predict(X_TEST)

model_xgb.fit(X_TRAIN, Y_TRAIN)
xgb_pred = model_xgb.predict(X_TEST)

model_lgb.fit(X_TRAIN, Y_TRAIN)
lgb_pred = model_lgb.predict(X_TEST)

ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15

In [24]:
print('***************** Sub Data *********************')
submission = pd.DataFrame(columns=['Id','Pred'])
submission['Id'] = feature_test[:,0]
submission['Pred'] = ensemble
submission.to_csv(path_test_out+ 'sub.csv',index=False)

***************** Sub Data *********************


In [28]:
stacked_pred

array([ 0.24022272,  0.31070304,  0.23135821,  0.23747067,  0.26796001,
        0.27344431,  0.28476498,  0.15569495,  0.30937957,  0.32759153,
        0.24620835,  0.30775418,  0.38620474,  0.33031222,  0.34389784,
        0.25330949,  0.28889351, -1.57563943,  0.24504857,  0.27705301,
        0.27526337,  0.35700989,  0.26499784,  0.24910781,  0.36501931,
        0.32092227,  0.285183  ,  0.25971224,  0.32638729,  0.32317649,
        0.34158582,  0.20874771,  0.28924394,  0.3040468 ,  0.24241204,
        0.31913866,  0.2966065 ,  0.36376503,  0.15953006,  0.30928753,
        0.30172799,  0.29276814,  0.28797251,  0.34873133,  0.29724858,
        0.73434002,  0.30672824,  0.62489285,  0.24903003,  0.30755574,
        0.30296364,  0.32630647,  0.25861381,  0.25761456,  0.29879566,
        2.85961103,  0.34787849,  0.24043941,  0.31713976,  0.05444074,
        0.04618315,  0.30974946,  0.24636576,  0.27415643,  0.24678817,
        0.24695171,  0.33710821,  0.2814967 ,  0.31460088, -0.01